In [3]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time

import os


# Import O-D Flow Data + Income Data:

In [14]:
str_1 = "027___403206_Redwood_City___off"
"Redwood_City" in str_1

True

In [15]:
def station_to_city(station_name):
    if "Palo_Alto" in station_name:
        city_name = "Palo Alto"
    elif "East_Palo_Alto" in station_name:
        city_name = "East Palo Alto"
    elif "Menlo_Park" in station_name:
        city_name = "Menlo Park"
    elif "Redwood_City" in station_name:
        city_name = "Redwood City"
    elif "Belmont" in station_name:
        city_name = "Belmont"
    elif "San_Mateo" in station_name:
        city_name = "San Mateo"
    elif "Burlingame" in station_name:
        city_name = "Burlingame"
    elif "Millbrae" in station_name:
        city_name = "Millbrae"
    else:
        assert 1 == 0, "There should be no other case."
    return city_name

In [16]:
directory_path = '../data/pems___101_N_Sep_2024/'
df_od_flow_data = pd.read_csv(directory_path + 'data_od.csv')

dict_df_od_flow_data = {}

for column_name_full in list(df_od_flow_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_od_flow_data[column_name_full].tolist()
    else:
        dict_df_od_flow_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start Index" or category == "End Index":
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = int(df_od_flow_data[column_name_full].tolist()[category_index])
            elif category == "O-D Flow (Initialization)" or category == "O-D Flow (Max Entropy)":
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = float(df_od_flow_data[column_name_full].tolist()[category_index])
            else:
                dict_df_od_flow_data[int(column_name_full)][category] \
                    = df_od_flow_data[column_name_full].tolist()[category_index]
        
        dict_df_od_flow_data[int(column_name_full)]["Start City"] \
            = station_to_city(dict_df_od_flow_data[int(column_name_full)]["Start Station"])
        dict_df_od_flow_data[int(column_name_full)]["End City"] \
            = station_to_city(dict_df_od_flow_data[int(column_name_full)]["End Station"])
        
# dict_df_od_flow_data
# dict_df_od_flow_data[0]["O-D Flow (Initialization)"]


617.3087999

In [25]:
directory_path = '../data/data_ACS_income/'
df_income_data = pd.read_csv(directory_path + 'data_income_raw___1_year_2023.csv')

dict_df_income_data = {}

for column_name_full in list(df_income_data.columns):
    if column_name_full == "Income":
        categories_list = df_income_data[column_name_full].tolist()
    else:
        dict_df_income_data[column_name_full] = {}
        for category_index, category in enumerate(categories_list):
            dict_df_income_data[column_name_full][category] \
                = float(df_income_data[column_name_full].tolist()[category_index])
                
# dict_df_income_data


# <font color='red'>From previous code:</font>

# Data Capping

In [ ]:
start_time = time.time()

directory_path = '../data/pems___101_N_Sep_2024/'
file_name_list = [f for f in os.listdir(directory_path) \
                  if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x' and f[0] != "~"]
file_name_list.sort()

thresh_lower = 0.1

# df = pd.read_excel(directory_path + file_name)
df_data_filtered = pd.read_csv(directory_path + 'data_filtered.csv', index_col=0)

dict_data_capped = {}
# dict_data_capped["Data Category"] = \
#     ["Station Type", "Number (data points)", "Mean (capped flow)", "Std Dev (capped flow)"]

times_dict = {}
for column_name_full in list(df_data_filtered.columns):
    print("column_name_full:", column_name_full)
    
    vals_list_str = df_data_filtered[column_name_full].tolist()
    
#     print("vals_list_str:", vals_list_str)

    column_name = column_name_full[:-5]
    
    dict_data_capped[column_name] = {}
    dict_data_capped[column_name]["Station Type"] = vals_list_str[0]
    
    vals_list = [float(val) for val in vals_list_str[1:]]
    max_val = max(vals_list)
    dict_data_capped[column_name]["Flows (capped)"] = \
        [val for val in vals_list if val != 0.0 and val >= max_val * thresh_lower]
    
    
#     print()
#     print("vals_list:", vals_list)
#     print()
#     print("vals_array_capped:", vals_array_capped)
    
#     dict_data_capped[column_name] += [vals_array_capped.shape[0]]
#     dict_data_capped[column_name] += [np.mean(vals_array_capped)]
#     dict_data_capped[column_name] += [np.std(vals_array_capped)]

    
    
#     print("dict_data_capped[column_name]:", dict_data_capped[column_name])
#     print()


end_time = time.time()

print()
print("Time:", end_time - start_time)



# Data Consolidation

# Scratch Work

In [ ]:
ell = [1, 2, 3, 4, 5]
# ell[1:-1]
ell += [6]
ell